In [1]:
from typing import Optional, Sequence, Tuple
from jax_party.env import JaxParty, PartyGenerator, PartyMARLWrapper
from jax_party.env_types import Action

import jax
import jax.numpy as jnp
import chex
import itertools
from jax_party.utils import tree_slice

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/ryanp/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def unpack_state(state):
    for key in state.keys():
        print(f"{key}: {state[key]}")

In [53]:
env = JaxParty(generator=PartyGenerator(), time_limit=4000)
env = PartyMARLWrapper(env)

key = jax.random.PRNGKey(0)
state, timestep = env.reset(key)
actions = jnp.array([-1, 0, 0])
state, ts = env.step(state, actions)
unpack_state(state)

active_agents: [0 1 1]
cumulative_rewards: [0. 4. 2.]
ranking: [1 2 0]
step_count: 1
action_mask: [[False False]
 [ True  True]
 [ True  True]]
key: [1700909826 1576785425]


In [54]:
actions = jnp.array([0, 0, 0])
state, timestep = env.step(state, actions)
unpack_state(state)

active_agents: [0 1 1]
cumulative_rewards: [0. 6. 6.]
ranking: [1 2 0]
step_count: 2
action_mask: [[False False]
 [ True  True]
 [ True  True]]
key: [4260505492 1636762242]


In [ ]:

# for actions in jnp.array(list(itertools.permutations([0, 1, 2]))):
#     # print("active_agents", state.active_agents)
#     # print("actions", actions)
#     actions = env._get_valid_actions(actions, state.action_mask)
#     # print("corrected actions", actions)
#     # state, timestep = env.step(state, actions)
#     print(state.cumulative_rewards, state.ranking)
#     print("-" * 30)
#     break

In [4]:
timestep.observation

Observation(agents_view=Array([[0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32), action_mask=Array([[ True, False, False],
       [False,  True,  True],
       [False,  True,  True]], dtype=bool), step_count=None)

In [9]:
env.observation_spec.agents_view.shape

(9,)

In [8]:
timestep.reward

Array(0., dtype=float32)

In [5]:
env.observation_spec.generate_value()

Observation(agents_view=Array([False, False, False, False, False, False, False, False, False],      dtype=bool), action_mask=Array([False, False, False], dtype=bool), step_count=Array([0, 0, 0], dtype=int32))

In [ ]:
from mava.utils.checkpointing import Checkpointer

loaded_checkpoint = Checkpointer(
    model_name=config.logger.system_name,
    **config.logger.checkpointing.load_args,  # Other checkpoint args
)
# Restore the learner state from the checkpoint
restored_params, _ = loaded_checkpoint.restore_params(
    input_params=Params(actor_params, critic_params)
)